In [2]:
import pandas as pd

# Load dataset
file_path = "dataset_2.tsv"
df = pd.read_csv(file_path, sep="\t")

# Display basic info
print("Shape of dataset:", df.shape)
print("\nFirst 5 rows:\n", df.head())
print("\nColumn names:", df.columns.tolist())

Shape of dataset: (4622, 5)

First 5 rows:
    order_id  quantity                              item_name  \
0         1         1           Chips and Fresh Tomato Salsa   
1         1         1                                   Izze   
2         1         1                       Nantucket Nectar   
3         1         1  Chips and Tomatillo-Green Chili Salsa   
4         2         2                           Chicken Bowl   

                                  choice_description item_price  
0                                                NaN     $2.39   
1                                       [Clementine]     $3.39   
2                                            [Apple]     $3.39   
3                                                NaN     $2.39   
4  [Tomatillo-Red Chili Salsa (Hot), [Black Beans...    $16.98   

Column names: ['order_id', 'quantity', 'item_name', 'choice_description', 'item_price']


In [3]:
# Check missing values before handling
print("Missing values before handling:\n")
print(df.isnull().sum())

# ---- Handling Missing Values ----

# 1. Drop rows where Order ID, Item Name, or Item Price are missing
df = df.dropna(subset=["order_id", "item_name", "item_price"])

# 2. Handle Quantity: if missing, fill with 1 (minimum sensible value)
if df["quantity"].isnull().sum() > 0:
    df["quantity"] = df["quantity"].fillna(1)

# 3. Handle Choice Description: fill missing with "No Choice"
df["choice_description"] = df["choice_description"].fillna("No Choice")

# Check missing values after handling
print("\nMissing values after handling:\n")
print(df.isnull().sum())

Missing values before handling:

order_id                 0
quantity                 0
item_name                0
choice_description    1246
item_price               0
dtype: int64

Missing values after handling:

order_id              0
quantity              0
item_name             0
choice_description    0
item_price            0
dtype: int64


In [4]:
# Display data types
print("Data types of each column:\n")
print(df.dtypes)

# Display a quick sample to inspect formatting issues
print("\nSample rows:\n", df.head())

# Ensure correct data types

# Convert order_id to integer
df["order_id"] = df["order_id"].astype(int)

# Convert quantity to integer
df["quantity"] = df["quantity"].astype(int)

# item_name and choice_description → string
df["item_name"] = df["item_name"].astype(str)
df["choice_description"] = df["choice_description"].astype(str)

# Clean and convert item_price: remove "$" and convert to float
df["item_price"] = df["item_price"].str.replace("$", "", regex=False).astype(float)

print("\nData types after adjustment:\n")
print(df.dtypes)

Data types of each column:

order_id               int64
quantity               int64
item_name             object
choice_description    object
item_price            object
dtype: object

Sample rows:
    order_id  quantity                              item_name  \
0         1         1           Chips and Fresh Tomato Salsa   
1         1         1                                   Izze   
2         1         1                       Nantucket Nectar   
3         1         1  Chips and Tomatillo-Green Chili Salsa   
4         2         2                           Chicken Bowl   

                                  choice_description item_price  
0                                          No Choice     $2.39   
1                                       [Clementine]     $3.39   
2                                            [Apple]     $3.39   
3                                          No Choice     $2.39   
4  [Tomatillo-Red Chili Salsa (Hot), [Black Beans...    $16.98   

Data types after

In [5]:
# 1. Check for duplicates
duplicates_count = df.duplicated().sum()
print(f"Number of fully duplicated rows: {duplicates_count}")

# 2. View some duplicate rows (if any)
if duplicates_count > 0:
    print("\nSample duplicated rows:\n")
    print(df[df.duplicated()].head())

# 3. Drop duplicates
df = df.drop_duplicates()

# 4. Verify
print(f"\nNumber of rows after removing duplicates: {len(df)}")

Number of fully duplicated rows: 59

Sample duplicated rows:

     order_id  quantity          item_name  \
238       103         1      Steak Burrito   
248       108         1        Canned Soda   
297       129         1      Steak Burrito   
381       165         1  Canned Soft Drink   
484       205         1       Chicken Bowl   

                                    choice_description  item_price  
238  [Tomatillo Red Chili Salsa, [Rice, Black Beans...       11.75  
248                                     [Mountain Dew]        1.09  
297  [Tomatillo Green Chili Salsa, [Rice, Cheese, G...       11.75  
381                                             [Coke]        1.25  
484  [Fresh Tomato Salsa, [Fajita Vegetables, Rice,...        8.75  

Number of rows after removing duplicates: 4563


In [8]:
# -------------------------------
# Step 1: Clean item_price column
# -------------------------------
# Ensure string, remove '$', strip spaces
df["item_price"] = (
    df["item_price"]
    .astype(str)                       # force string
    .str.replace("$", "", regex=False) # remove $
    .str.strip()                       # remove spaces
)

# Convert to numeric (invalid parsing -> NaN)
df["item_price"] = pd.to_numeric(df["item_price"], errors="coerce")

# Drop rows where item_price couldn't be converted
df = df.dropna(subset=["item_price"])

# -------------------------------
# Step 2: Examine Quantity & Price
# -------------------------------
print("Quantity column statistics:\n", df["quantity"].describe())
print("\nItem Price column statistics:\n", df["item_price"].describe())

# Unique values check
print("\nUnique quantities (sample):", df["quantity"].unique()[:20])
print("\nUnique item prices (sample):", df["item_price"].unique()[:20])

# -------------------------------
# Step 3: Handle anomalies
# -------------------------------
# Remove rows with invalid quantities (<= 0)
df = df[df["quantity"] > 0]

# Remove rows with invalid prices (<= 0)
df = df[df["item_price"] > 0]

# Optional: cap extreme prices (example: > $100 seems unrealistic)
# df = df[df["item_price"] <= 100]

# -------------------------------
# Step 4: Create total_price column
# -------------------------------
df["total_price"] = df["quantity"] * df["item_price"]

print("\nAfter cleaning:")
print("Min quantity:", df["quantity"].min())
print("Min item_price:", df["item_price"].min())
print("\nFirst 5 rows:\n", df.head())

Quantity column statistics:
 count    4563.000000
mean        1.076704
std         0.412739
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        15.000000
Name: quantity, dtype: float64

Item Price column statistics:
 count    4563.000000
mean        7.490083
std         4.244155
min         1.090000
25%         3.750000
50%         8.750000
75%         9.250000
max        44.250000
Name: item_price, dtype: float64

Unique quantities (sample): [ 1  2  3  4  5 15  7  8 10]

Unique item prices (sample): [ 2.39  3.39 16.98 10.98  1.69 11.75  9.25  4.45  8.75 11.25  8.49  2.18
  8.99  1.09  2.95  2.15  3.99 22.5  11.48 17.98]

After cleaning:
Min quantity: 1
Min item_price: 1.09

First 5 rows:
    order_id  quantity                              item_name  \
0         1         1           Chips and Fresh Tomato Salsa   
1         1         1                                   Izze   
2         1         1                       Nantucket Nectar   
3 

In [9]:
# Check unique values and missing count
print("Missing values in choice_description:", df["choice_description"].isnull().sum())
print("\nNumber of unique choice descriptions:", df["choice_description"].nunique())

# Sample values
print("\nSample choice descriptions:\n", df["choice_description"].unique()[:10])

# Fill missing with "No Choice"
df["choice_description"] = df["choice_description"].fillna("No Choice")

# Remove square brackets and extra spaces
df["choice_description"] = (
    df["choice_description"]
    .str.replace("[", "", regex=False)
    .str.replace("]", "", regex=False)
    .str.strip()
)

# Split multiple choices into a list
df["choice_list"] = df["choice_description"].str.split(", ")

# Example: show the first few
print(df[["item_name", "choice_description", "choice_list"]].head(10))

Missing values in choice_description: 0

Number of unique choice descriptions: 1044

Sample choice descriptions:
 ['No Choice' '[Clementine]' '[Apple]'
 '[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]'
 '[Fresh Tomato Salsa (Mild), [Rice, Cheese, Sour Cream, Guacamole, Lettuce]]'
 '[Tomatillo Red Chili Salsa, [Fajita Vegetables, Black Beans, Pinto Beans, Cheese, Sour Cream, Guacamole, Lettuce]]'
 '[Tomatillo Green Chili Salsa, [Pinto Beans, Cheese, Sour Cream, Lettuce]]'
 '[Fresh Tomato Salsa, [Rice, Black Beans, Pinto Beans, Cheese, Sour Cream, Lettuce]]'
 '[Roasted Chili Corn Salsa, [Fajita Vegetables, Rice, Black Beans, Cheese, Sour Cream]]'
 '[Roasted Chili Corn Salsa, [Rice, Black Beans, Cheese, Sour Cream]]']
                               item_name  \
0           Chips and Fresh Tomato Salsa   
1                                   Izze   
2                       Nantucket Nectar   
3  Chips and Tomatillo-Green Chili Salsa   
4                          

In [10]:
import re

# Function to check special characters
def find_special_chars(series):
    special_chars = series.str.findall(r"[^a-zA-Z0-9\s]").explode().dropna().unique()
    return special_chars

print("Special characters in item_name:", find_special_chars(df["item_name"]))
print("Special characters in choice_description:", find_special_chars(df["choice_description"]))

# Remove brackets and extra spaces from choice_description
df["choice_description"] = (
    df["choice_description"]
    .str.replace(r"[\[\]]", "", regex=True)  # remove square brackets
    .str.replace(r"\s+", " ", regex=True)   # replace multiple spaces with one
    .str.strip()
)

# Standardize item_name (remove extra spaces, unify casing)
df["item_name"] = (
    df["item_name"]
    .str.replace(r"\s+", " ", regex=True)  # fix spacing
    .str.strip()
    .str.title()                           # Title Case for consistency
)

# (Optional) lowercase everything if needed
# df["item_name"] = df["item_name"].str.lower()
# df["choice_description"] = df["choice_description"].str.lower()

Special characters in item_name: ['-']
Special characters in choice_description: ['-' '(' ')' ',' '.']


In [11]:
# Order ID stats
print("Order ID column type:", df["order_id"].dtype)
print("Total unique order IDs:", df["order_id"].nunique())
print("Total rows:", len(df))

# Check min and max
print("Order ID range:", df["order_id"].min(), "to", df["order_id"].max())

# Check for missing values
print("Missing Order IDs:", df["order_id"].isnull().sum())

# Check if order_id is strictly increasing or has gaps
expected_count = df["order_id"].max() - df["order_id"].min() + 1
actual_count = df["order_id"].nunique()
print("Expected count:", expected_count, "| Actual unique:", actual_count)
print("Missing order IDs:", expected_count - actual_count)

# How many items per order
order_sizes = df.groupby("order_id")["item_name"].count()

print("\nOrder size stats:")
print(order_sizes.describe())

# Example: show largest orders
print("\nTop 5 biggest orders:\n", order_sizes.sort_values(ascending=False).head())

Order ID column type: int64
Total unique order IDs: 1834
Total rows: 4563
Order ID range: 1 to 1834
Missing Order IDs: 0
Expected count: 1834 | Actual unique: 1834
Missing order IDs: 0

Order size stats:
count    1834.000000
mean        2.488004
std         1.154284
min         1.000000
25%         2.000000
50%         2.000000
75%         3.000000
max        21.000000
Name: item_name, dtype: float64

Top 5 biggest orders:
 order_id
926     21
1483    14
1786    11
205     11
759     11
Name: item_name, dtype: int64


In [12]:
# Unique item names count
print("Unique item names:", df["item_name"].nunique())

# Sample unique names
print("\nSample item names:\n", df["item_name"].unique()[:20])

# Look for potential duplicates caused by case/spacing
print("\nUnique names (lowercased):", df["item_name"].str.lower().nunique())

df["item_name"] = (
    df["item_name"]
    .str.strip()                       # remove leading/trailing spaces
    .str.replace(r"\s+", " ", regex=True)  # replace multiple spaces with one
    .str.title()                       # Title Case for uniformity
)

# Check items sorted alphabetically to spot similar variations
for name in sorted(df["item_name"].unique())[:50]:
    print(name)

# Optional: fuzzy matching for close duplicates
from difflib import get_close_matches

unique_items = df["item_name"].unique().tolist()
print("\nExamples of close matches to 'Chicken Bowl':")
print(get_close_matches("Chicken Bowl", unique_items, n=5, cutoff=0.8))

# Example mapping dictionary
mapping = {
    "Chips And Guacamole": "Chips & Guacamole",
    "Chicken Bowl With Chips": "Chicken Bowl"
}

df["item_name"] = df["item_name"].replace(mapping)

Unique item names: 50

Sample item names:
 ['Chips And Fresh Tomato Salsa' 'Izze' 'Nantucket Nectar'
 'Chips And Tomatillo-Green Chili Salsa' 'Chicken Bowl' 'Side Of Chips'
 'Steak Burrito' 'Steak Soft Tacos' 'Chips And Guacamole'
 'Chicken Crispy Tacos' 'Chicken Soft Tacos' 'Chicken Burrito'
 'Canned Soda' 'Barbacoa Burrito' 'Carnitas Burrito' 'Carnitas Bowl'
 'Bottled Water' 'Chips And Tomatillo Green Chili Salsa' 'Barbacoa Bowl'
 'Chips']

Unique names (lowercased): 50
6 Pack Soft Drink
Barbacoa Bowl
Barbacoa Burrito
Barbacoa Crispy Tacos
Barbacoa Salad Bowl
Barbacoa Soft Tacos
Bottled Water
Bowl
Burrito
Canned Soda
Canned Soft Drink
Carnitas Bowl
Carnitas Burrito
Carnitas Crispy Tacos
Carnitas Salad
Carnitas Salad Bowl
Carnitas Soft Tacos
Chicken Bowl
Chicken Burrito
Chicken Crispy Tacos
Chicken Salad
Chicken Salad Bowl
Chicken Soft Tacos
Chips
Chips And Fresh Tomato Salsa
Chips And Guacamole
Chips And Mild Fresh Tomato Salsa
Chips And Roasted Chili Corn Salsa
Chips And Roasted Chi

In [13]:
# Summary stats
print("Quantity stats:\n", df["quantity"].describe())
print("\nItem Price stats:\n", df["item_price"].describe())

# Check relationship between quantity and item_price
df["unit_price"] = df["item_price"] / df["quantity"]

print("\nUnit price stats:\n", df["unit_price"].describe())

# Look at some high-quantity orders
print("\nSample high-quantity rows:\n", df[df["quantity"] > 5][["quantity", "item_price", "unit_price", "item_name"]].head(10))

# Find anomalies where unit price looks unrealistic
invalid_rows = df[(df["unit_price"] <= 0) | (df["unit_price"] > 100)]
print("\nPotential anomalies:\n", invalid_rows.head())

# (Optional) Remove them
df = df[~df.index.isin(invalid_rows.index)]

Quantity stats:
 count    4563.000000
mean        1.076704
std         0.412739
min         1.000000
25%         1.000000
50%         1.000000
75%         1.000000
max        15.000000
Name: quantity, dtype: float64

Item Price stats:
 count    4563.000000
mean        7.490083
std         4.244155
min         1.090000
25%         3.750000
50%         8.750000
75%         9.250000
max        44.250000
Name: item_price, dtype: float64

Unit price stats:
 count    4563.000000
mean        7.105260
std         3.658207
min         1.090000
25%         2.950000
50%         8.750000
75%         9.250000
max        11.890000
Name: unit_price, dtype: float64

Sample high-quantity rows:
       quantity  item_price  unit_price                     item_name
3598        15       44.25        2.95  Chips And Fresh Tomato Salsa
3599         7       10.50        1.50                 Bottled Water
3887         8       13.52        1.69                 Side Of Chips
4152        10       15.00        1.5

In [14]:
# Compute per-unit price
df["unit_price"] = df["item_price"] / df["quantity"]

print("Unit price stats:\n", df["unit_price"].describe())

# Average unit price per item
item_prices = df.groupby("item_name")["unit_price"].agg(["min", "max", "mean", "nunique"]).reset_index()

# Items with multiple different unit prices
price_inconsistencies = item_prices[item_prices["nunique"] > 1]

print("\nItems with inconsistent unit prices:\n", price_inconsistencies.head(20))
# Example: look at one inconsistent item
example_item = "Chicken Burrito"
print(df[df["item_name"] == example_item][["quantity", "item_price", "unit_price", "choice_description"]].head(15))

# Check for same item + same choice having multiple unit prices
dup_check = (
    df.groupby(["item_name", "choice_description"])["unit_price"]
    .nunique()
    .reset_index()
)

# Filter anomalies (same item & choice but >1 unique price)
true_anomalies = dup_check[dup_check["unit_price"] > 1]
print("\nTrue anomalies (same item & choice but multiple prices):\n", true_anomalies)

Unit price stats:
 count    4563.000000
mean        7.105260
std         3.658207
min         1.090000
25%         2.950000
50%         8.750000
75%         9.250000
max        11.890000
Name: unit_price, dtype: float64

Items with inconsistent unit prices:
                 item_name   min    max       mean  nunique
1           Barbacoa Bowl  8.69  11.75  10.201692        6
2        Barbacoa Burrito  8.69  11.75   9.838889        6
3   Barbacoa Crispy Tacos  8.99  11.75  10.087273        4
4     Barbacoa Salad Bowl  9.39  11.89  10.778889        2
5     Barbacoa Soft Tacos  8.99  11.75  10.018400        4
6           Bottled Water  1.09   1.50   1.428581        2
7                    Bowl  7.40   7.40   7.400000        2
11          Carnitas Bowl  8.99  11.75  10.315588        5
12       Carnitas Burrito  8.69  11.75   9.975932        6
13  Carnitas Crispy Tacos  8.99  11.75   9.852857        3
15    Carnitas Salad Bowl  9.39  11.89  11.056667        2
16    Carnitas Soft Tacos  8.99  

In [15]:
# Save cleaned dataset to CSV
output_file = "dataset_cleaned_2.csv"
df.to_csv(output_file, sep=",", index=False)

print(f"Cleaned dataset has been saved to: {output_file}")


Cleaned dataset has been saved to: dataset_cleaned_2.csv
